In [26]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from pyquadkey2 import quadkey
from shapely.geometry import Point, Polygon, MultiPolygon

In [27]:


def get_point_in_polygon(lat, lon, polygons):
    """
    @param lat: double
    @param lon: double
    @param polygons: dict
    @return geo_id: str
    """
    point = Point(lon, lat)
    for geo_id in polygons:
        polygon = polygons[geo_id]
        if isinstance(polygon, Polygon):
            if polygon.contains(point):
                return geo_id
        elif isinstance(polygon, MultiPolygon):
            for subpolygon in polygon.geoms:
                if subpolygon.contains(point):
                    return geo_id
    return 'null'


In [28]:
shapefile = gpd.read_file('/Users/kismatkhatri/Documents/Capstone project/shapefile/pak_admbnda_adm2_wfp_20220909.shp')
polygons = dict(zip(shapefile['ADM2_PCODE'], shapefile['geometry']))

print(shapefile.shape)
shapefile.head()

(160, 15)


,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,1.594116,0.067758,Bagh,PK101,None,None,None,Azad Kashmir,PK1,Pakistan,PK,2022-09-02,2022-09-09,None,"POLYGON ((73.53892 34.12617, 73.53952 34.12492..."
1,1.987888,0.117047,Bhimber,PK102,None,None,None,Azad Kashmir,PK1,Pakistan,PK,2022-09-02,2022-09-09,None,"POLYGON ((73.96443 33.24121, 73.96562 33.24077..."
2,1.300416,0.066683,Jhelum Valley,PK103,None,None,None,Azad Kashmir,PK1,Pakistan,PK,2022-09-02,2022-09-09,None,"POLYGON ((73.91993 34.34231, 73.94097 34.30174..."
3,1.001545,0.053722,Haveli,PK104,None,None,None,Azad Kashmir,PK1,Pakistan,PK,2022-09-02,2022-09-09,None,"POLYGON ((74.15687 34.04042, 74.16142 34.03887..."
4,2.017824,0.155069,Kotli,PK105,None,None,None,Azad Kashmir,PK1,Pakistan,PK,2022-09-02,2022-09-09,None,"POLYGON ((73.62197 33.65854, 73.62265 33.65844..."


In [29]:
rwi_df = pd.read_csv('/Users/kismatkhatri/Documents/Capstone project/joinedRWI_population.csv')
rwi_df.head()


,latitude,longitude,rwi,error,quadkey,population
0,33.897776,70.037842,-0.074,0.624,12310221130231,1190.728087
1,31.118794,66.807861,-0.569,0.368,12303111302200,99.496179
2,32.648625,73.245850,-0.193,0.498,12310322022121,5348.915150
3,35.182788,72.894287,-0.178,0.355,12310213112323,773.519400
4,25.948166,69.268799,-0.530,0.450,12312201232000,728.985542


In [30]:
rwi_df['geo_id'] = rwi_df.apply(lambda x: get_point_in_polygon(x['latitude'], x['longitude'], polygons), axis=1)
rwi_df = rwi_df[rwi_df['geo_id'] != 'null']

print(rwi_df.shape)
rwi_df.head()
 

(85643, 7)


,latitude,longitude,rwi,error,quadkey,population,geo_id
0,33.897776,70.037842,-0.074,0.624,12310221130231,1190.728087,PK518
1,31.118794,66.807861,-0.569,0.368,12303111302200,99.496179,PK235
2,32.648625,73.245850,-0.193,0.498,12310322022121,5348.915150,PK613
3,35.182788,72.894287,-0.178,0.355,12310213112323,773.519400,PK515
4,25.948166,69.268799,-0.530,0.450,12312201232000,728.985542,PK718


In [31]:
rwi_df.to_csv('/Users/kismatkhatri/Documents/Capstone project/RWI_population_quadkey_geo_id.csv', index=False)

In [33]:
bing_tile_z14_pop = rwi_df.groupby('quadkey', as_index=False)['population'].sum()


In [34]:
geo_pop = rwi_df.groupby('geo_id', as_index=False)['population'].sum()


In [35]:
rwi_df = rwi_df.merge(bing_tile_z14_pop, on='quadkey', how='left')


In [37]:
rwi_df.columns

Index(['latitude', 'longitude', 'rwi', 'error', 'quadkey', 'population_x',
       'geo_id', 'population_y'],
      dtype='object')

In [39]:
rwi_df['population_x']

0        1190.728087
1          99.496179
2        5348.915150
3         773.519400
4         728.985542
            ...     
85638     122.458884
85639    4363.553928
85640     201.658053
85641     267.999888
85642    3443.343820
Name: population_x, Length: 85643, dtype: float64

In [40]:
rwi_df['population_y']

0        1190.728087
1          99.496179
2        5348.915150
3         773.519400
4         728.985542
            ...     
85638     122.458884
85639    4363.553928
85640     201.658053
85641     535.999776
85642    6886.687640
Name: population_y, Length: 85643, dtype: float64

In [38]:
geo_pop.columns

Index(['geo_id', 'population'], dtype='object')

In [41]:
rwi_df['pop_weight'] = rwi_df['population_y'] / rwi_df['geo_id'].map(geo_pop.set_index('geo_id')['population'])
rwi_df['rwi_weight'] = rwi_df['rwi'] * rwi_df['pop_weight']


In [42]:
rwi_df['rwi_weight']

0       -0.000128
1       -0.000293
2       -0.000675
3       -0.000641
4       -0.000181
           ...   
85638   -0.000169
85639    0.000701
85640   -0.000044
85641   -0.000061
85642   -0.002611
Name: rwi_weight, Length: 85643, dtype: float64

In [44]:
geo_rwi = rwi_df.groupby('geo_id', as_index=False)['rwi_weight'].sum()


In [46]:
geo_rwi['rwi_weight']

0      0.050482
1      0.027296
2      0.021551
3     -0.263393
4      0.066787
         ...   
155   -0.249242
156   -0.347638
157   -0.250791
158   -0.196405
159    0.889178
Name: rwi_weight, Length: 160, dtype: float64